In [61]:
# import libraries 
import pandas as pd
import numpy as np
import math
from numpy.linalg import norm

from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from pickle import dump, load

pd.set_option('display.max_columns', None)

np.random_state = 42

## Yüklüyoruz `sd_pp` Dataset

In [62]:
# load in df with the cluster labels
sd_pp = pd.read_csv('data/sd_pp', index_col = 0)
sd_pp

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0,-1.152178,-0.561234,-0.533910,-0.728205,2.546221,0.605242,1.504821,-0.909452,-0.563027,-0.490463,3.516095,-0.408407,0.513198,-0.764114,-0.238612,0.038199,-0.972913,-0.581096,-0.581096,0.372801,0.368450,0.464503,0.272007,0.278657,0.342548,0.598151,0.0,0.970377,0.0,-0.167773,-0.183102
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

### Get the `listing_url`

In [63]:
# load in df with the listing urls
sd_listings_url = pd.read_csv('data/sd_modeling_with_urls', index_col= 0)
sd_listings_url = sd_listings_url[['listing_url']]
sd_listings_url = sd_listings_url.reset_index(drop = 'index')
sd_listings_url

,listing_url
0,https://www.airbnb.com/rooms/33159143
1,https://www.airbnb.com/rooms/17138468
2,https://www.airbnb.com/rooms/21898446
3,https://www.airbnb.com/rooms/25948680
4,https://www.airbnb.com/rooms/1756516
...,...
13035,https://www.airbnb.com/rooms/20473217
13036,https://www.airbnb.com/rooms/17975465
13037,https://www.airbnb.com/rooms/4227110
13038,https://www.airbnb.com/rooms/17975942


In [64]:
# save as csv
path = "deployment/"

sd_listings_url.to_csv(path + 'url_listings')

In [65]:
import os
import pandas as pd

# Örnek bir DataFrame oluşturma (sd_listings_url değişkeninin tanımlı olduğunu varsayıyorum)
# sd_listings_url = pd.DataFrame(...)  # Mevcut DataFrame'inizi burada tanımlayın

# Dosya yolu
path = "deployment/"

# Dizin mevcut değilse oluştur
if not os.path.exists(path):
    os.makedirs(path)

# CSV dosyasını kaydetme
sd_listings_url.to_csv(path + 'url_listings.csv', index=False)  # 'index=False' ile index kaydını dahil etmez


## Karşılaştırma listesi için yeni bir veri çerçevesi oluştur

**Select a listing**

In [66]:
# sd_clustered veri çerçevesinden rastgele bir listeleme seçer
random_listing = sd_pp.sample(n = 1)
random_listing

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
3185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,-0.819868,-0.561234,-0.53391,-0.728205,-0.504891,-0.460091,-0.55543,-0.670653,-0.563027,-0.490463,-0.175004,0.389204,0.513198,1.308706,-0.23291,0.038199,-0.972913,-0.33123,-0.33123,0.527926,0.36845,0.464503,0.272007,0.278657,-1.583221,0.598151,0.0,-1.030527,0.0,-0.167773,-0.183102


###  Arraye DÖnüştür

In [67]:
# Tek bir listeyi diziye dönüştür.
random_listing_array = random_listing.values

# Tüm listeyi diziye dönüştür.
sd_pp_array = sd_pp.values

In [68]:
print(random_listing_array.shape)
print(sd_pp_array.shape) 

(1, 240)
(13040, 240)


## Cosine Similarity

In [69]:
# define two lists or arrays to compare
A = np.squeeze(np.asarray(sd_pp_array))
B = np.squeeze(np.asarray(random_listing_array))
print("A:\n", A)
print("B:\n", B)
 
# Kosinüs benzerliğini hesapla
cosine = np.dot(A,B)/(norm(A, axis=1)*norm(B))
print("Cosine Similarity:\n", cosine)

A:
 [[ 0.         0.         0.        ...  0.        -0.1677734 -0.1831016]
 [ 0.         0.         0.        ...  0.        -0.1677734 -0.1831016]
 [ 0.         0.         0.        ...  0.        -0.1677734 -0.1831016]
 ...
 [ 0.         0.         0.        ...  0.        -0.1677734 -0.1831016]
 [ 0.         0.         0.        ...  0.        -0.1677734 -0.1831016]
 [ 0.         0.         0.        ...  0.        -0.1677734 -0.1831016]]
B:
 [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          1.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.

### Kosinüs Benzerliğini `sd_clustered`veri çerçevesine ekle

In [70]:
# load in df with the cluster labels
sd_clustered = pd.read_csv('data/sd_clustered', index_col = 0)
sd_clustered.reset_index(drop = 'index')

,id,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
0,33159143,1,32.91736,-117.07635,Scripps Ranch,92131,House,Private room,1,1.0,1.0,1,Real Bed,1100.0,1100.0,1000.0,0.0,1,0.0,60,90,within a few hours,100.0,0,1.0,1,0,0,0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0,1,0,strict_14_with_grace_period,0,0
1,17138468,4,32.84067,-117.27443,La Jolla,92037,Apartment,Entire home/apt,1,2.0,2.0,3,Real Bed,1400.0,1400.0,0.0,0.0,1,0.0,30,1125,no response,0.0,0,1.0,1,0,2,4,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0,1,0,strict_14_with_grace_period,0,0
2,21898446,4,32.79797,-117.24250,Pacific Beach,92109,Townhouse,Private room,1,1.0,1.0,1,Real Bed,1250.0,1250.0,0.0,0.0,1,0.0,30,30,no response,0.0,0,1.0,1,0,0,0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0,1,0,flexible,0,0
3,25948680,4,32.77545,-117.05923,College Area,92120,Apartment,Entire home/apt,1,1.0,1.0,1,Real Bed,1150.0,1150.0,0.0,0.0,1,0.0,31,40,no response,0.0,0,1.0,1,0,0,0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,flexible,0,0
4,1756516,0,32.84619,-117.27558,La Jolla,92037,Condominium,Private room,1,1.0,1.0,1,Real Bed,110.0,110.0,200.0,0.0,1,50.0,180,365,within a day,67.0,0,6.0,1,1,2,4,90.0,7.0,7.0,8.0,8.0,9.0,10.0,0,0,0,strict_14_with_grace_period,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13035,20473217,2,32.78399,-117.25391,Mission Bay,92109,Villa,Entire home/apt,20,7.0,8.0,11,Real Bed,3000.0,3000.0,3000.0,0.0,20,0.0,1,1125,within an hour,95.0,1,86.0,1,1,0,0,96.0,10.0,10.0,10.0,10.0,10.0,9.0,0,0,0,luxury_moderate,0,0
13036,17975465,2,32.80892,-117.26588,La Jolla,92037,House,Entire home/apt,20,7.0,10.0,13,Real Bed,2823.0,2823.0,1000.0,874.0,20,0.0,1,1125,within an hour,99.0,0,403.0,1,0,6,12,83.0,8.0,8.0,9.0,9.0,10.0,8.0,0,1,0,super_strict_60,0,0
13037,4227110,2,32.79395,-117.23492,Mission Bay,92109,House,Entire home/apt,20,2.0,4.0,10,Real Bed,208.0,208.0,1000.0,332.0,4,25.0,1,1125,within an hour,96.0,0,11.0,1,1,142,284,92.0,9.0,9.0,9.0,9.0,10.0,8.0,0,1,0,strict_14_with_grace_period,0,0
13038,17975942,2,32.83598,-117.27878,La Jolla,92037,House,Entire home/apt,21,5.5,7.0,13,Real Bed,1305.0,1305.0,1000.0,800.0,21,0.0,1,1125,within an hour,99.0,0,403.0,1,0,4,8,80.0,9.0,8.0,10.0,8.0,10.0,9.0,0,1,0,super_strict_60,0,0


#### Adding in `listing_url`

In [71]:
# merge on index
sd_clustered = sd_clustered.join(sd_listings_url)

In [72]:
# add similarity 
rec = sd_clustered.copy()
rec['similarity'] = pd.DataFrame(cosine).values

In [73]:
# reorder column names
rec = rec[['id','listing_url', 'similarity', 'cluster_label', 'latitude', 'longitude',
       'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'number_of_reviews', 'number_of_stays',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']]

## Recommendations AL!

In [74]:
# En yüksek benzerliğe göre sıralayın.
rec.sort_values(by = ['similarity'], ascending = False).head(6)

,id,listing_url,similarity,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
3190,5005059,https://www.airbnb.com/rooms/7053189,1.000000,1,32.75432,-117.08127,City Heights East,92115,Condominium,Private room,2,1.0,1.0,1,Real Bed,50.0,100.0,0.0,25.0,1,0.0,2,1125,within an hour,100.0,1,2.0,1,0,15,30,99.0,10.0,10.0,10.0,10.0,9.0,10.0,0,0,0,strict_14_with_grace_period,0,0
1255,20835671,https://www.airbnb.com/rooms/28409599,0.916787,1,32.75439,-117.08120,City Heights East,92115,Condominium,Private room,2,1.0,1.0,1,Real Bed,69.0,483.0,0.0,0.0,1,0.0,7,1125,within an hour,100.0,1,2.0,1,0,1,2,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,strict_14_with_grace_period,0,0
3929,32727447,https://www.airbnb.com/rooms/37504406,0.829790,1,32.73973,-117.05635,Oak Park,92115,Townhouse,Private room,2,1.0,1.0,1,Real Bed,69.0,69.0,0.0,20.0,2,10.0,1,1125,within an hour,100.0,1,1.0,1,0,36,72,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,strict_14_with_grace_period,0,0
249,23238267,https://www.airbnb.com/rooms/28989556,0.826490,1,32.86171,-117.22626,University City,92122,Condominium,Private room,1,1.0,1.0,1,Real Bed,65.0,195.0,0.0,0.0,1,0.0,3,30,within an hour,100.0,1,1.0,1,0,26,52,99.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,strict_14_with_grace_period,0,0
2199,29436239,https://www.airbnb.com/rooms/36357669,0.823741,1,32.73404,-117.10770,City Heights West,92105,Guest suite,Entire home/apt,2,1.0,0.0,1,Real Bed,99.0,198.0,200.0,30.0,1,0.0,2,1125,within an hour,100.0,1,1.0,1,0,39,78,99.0,10.0,10.0,10.0,10.0,9.0,10.0,0,0,0,strict_14_with_grace_period,0,0
2883,26787733,https://www.airbnb.com/rooms/27992044,0.811395,1,32.74876,-117.11300,City Heights West,92105,Condominium,Private room,2,1.0,1.0,1,Real Bed,65.0,130.0,100.0,25.0,1,0.0,2,20,within an hour,100.0,1,1.0,1,0,18,36,100.0,10.0,10.0,10.0,10.0,9.0,10.0,0,0,0,moderate,0,0


### Get Recommendations Fonksiyonu

In [75]:
# `sd_clustered` veri setinden rastgele bir listeleme seçer.
random_listing = sd_pp.sample(n = 1)
random_listing

,Allied Gardens,Alta Vista,Amphitheater And Water Park,Balboa Park,Bario Logan,Bay Ho,Bay Park,Bay Terrace,Bird Land,Bonita Long Canyon,Carmel Mountain,Carmel Valley,Chollas View,City Heights East,City Heights West,Clairemont Mesa,College Area,Columbia,Core,Cortez Hill,Darnall,Del Cerro,Del Mar Heights,East Lake,East Village,Eastlake Trails,Eastlake Vistas,Eastlake Woods,Egger Highlands,El Cerritos,Emerald Hills,Encanto,Estlake Greens,Gaslamp Quarter,Gateway,Grant Hill,Grantville,Horton Plaza,Jomacha-Lomita,Kearny Mesa,Kensington,La Jolla,La Jolla Village,Lake Murray,Lincoln Park,Linda Vista,Little Italy,Loma Portal,Lynwood Hills,Marina,Memorial,Midtown,Midtown District,Mira Mesa,Mission Bay,Mission Valley,Moreno Mission,Mount Hope,Mountain View,Nestor,Normal Heights,North City,North Clairemont,North Hills,Northwest,Oak Park,Ocean Beach,Old Town,Otay Ranch,Pacific Beach,Palm City,Paradise Hills,Park West,Paseo Ranchoero,Rancho Bernadino,Rancho Del Rey,Rancho Penasquitos,Rolando,Rolling Hills Ranch,Roseville,Sabre Springs,San Carlos,San Ysidro,Scripps Ranch,Serra Mesa,Sky Line,Sorrento Valley,South Park,Southcrest,Southwest,Sunbow,Talmadge,Terra Nova,Thomy Locust Pl,Tierrasanta,Tijuana River Valley,Torrey Pines,University City,Valencia Park,Webster,West University Heights,Wooded Area,Yosemite Dr,22000,22010,22425,22435,91901,91902,91910,91911,91913,91914,91915,91932,91941,91942,91945,91950,92014,92025,92029,92037,92054,92064,92071,92075,92101,92102,92103,92104,92105,92106,92107,92108,92109,92110,92111,92113,92114,92115,92116,92117,92118,92119,92120,92121,92122,92123,92124,92126,92127,92128,92129,92130,92131,92139,92154,92173,92307,92618,921096,Aparthotel,Apartment,Barn,Bed and breakfast,Boat,Boutique hotel,Bungalow,Bus,Cabin,Camper/RV,Campsite,Casa particular (Cuba),Castle,Cave,Chalet,Condominium,Cottage,Dome house,Earth house,Farm stay,Guest suite,Guesthouse,Hostel,Hotel,House,Igloo,Loft,Nature lodge,Other,Resort,Serviced apartment,Tent,Tiny house,Townhouse,Treehouse,Vacation home,Villa,Entire home/apt,Private room,Shared room,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
10173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.509372,-0.561234,0.302992,1.902354,-0.173627,-0.217195,0.060585,-0.049773,0.728419,0.596054,-0.175004,0.389204,0.513198,-0.764114,-0.215804,0.038199,-0.972913,-0.247942,-0.247942,-0.713077,-1.157784,0.464503,-1.628837,0.278657,0.342548,-0.717619,0.0,0.970377,0.0,-0.167773,-0.183102


In [76]:
def get_recommendations(df, listing):
    """
    Ön işlenmiş veri çerçevesi ve seçilen listelemeyi girdi olarak alır
      ve kosinüs benzerliğine dayalı olarak ilk 5 (listeleme dahil) öneriyi sunar.
    """
    # reset the index
    df = df.reset_index(drop = 'index')
    
    # Tek bir listelemeyi diziye dönüştür.
    listing_array = listing.values

    # convert all listings to an array
    df_array = df.values
    
    # Dizileri tek boyutlu hale getirin.
    A = np.squeeze(np.asarray(df_array))
    B = np.squeeze(np.asarray(listing_array))
    
    # Kosinüs benzerliğini hesapla
    cosine = np.dot(A,B)/(norm(A, axis = 1)*norm(B))
    
    # Benzerliği öneriler veri çerçevesine ekle ve indeks sıfırla
    rec = sd_clustered.copy().reset_index(drop = 'index')
    rec['similarity'] = pd.DataFrame(cosine).values
    
    # reorder column names
    rec = rec[['id','listing_url', 'similarity', 'cluster_label', 'latitude', 'longitude',
       'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'nightly_price', 'price_per_stay', 'security_deposit', 'cleaning_fee',
       'guests_included', 'extra_people', 'minimum_nights', 'maximum_nights',
       'host_response_time', 'host_response_rate', 'host_is_superhost',
       'host_total_listings_count', 'host_has_profile_pic',
       'host_identity_verified', 'number_of_reviews', 'number_of_stays',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'cancellation_policy',
       'require_guest_profile_picture', 'require_guest_phone_verification']]
    
    # sort by top 5 descending
    return rec.sort_values(by = ['similarity'], ascending = False).head(6)


In [77]:
get_recommendations(sd_pp, random_listing)

,id,listing_url,similarity,cluster_label,latitude,longitude,neighbourhood_cleansed,zipcode,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_time,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification
10173,25091961,https://www.airbnb.com/rooms/18852697,1.000000,3,32.71491,-117.15455,East Village,92101,Loft,Entire home/apt,6,1.0,2.0,6,Real Bed,164.0,328.0,299.0,90.0,4,30.0,2,1125,within an hour,100.0,0,5.0,1,0,20,40,91.0,9.0,10.0,9.0,10.0,10.0,9.0,0,1,0,strict_14_with_grace_period,0,0
11777,34144931,https://www.airbnb.com/rooms/28080657,0.858235,3,32.71100,-117.15947,Gaslamp Quarter,92101,Apartment,Entire home/apt,8,2.0,2.0,5,Real Bed,164.0,328.0,199.0,100.0,4,25.0,2,1125,within an hour,100.0,0,5.0,1,0,13,26,91.0,9.0,10.0,9.0,10.0,10.0,10.0,0,1,0,strict_14_with_grace_period,0,0
10139,30574647,https://www.airbnb.com/rooms/20672946,0.853787,0,32.71146,-117.15805,East Village,92101,Loft,Entire home/apt,6,1.0,2.0,6,Real Bed,175.0,350.0,199.0,90.0,4,30.0,2,1125,within an hour,100.0,0,8.0,1,0,13,26,85.0,8.0,10.0,8.0,9.0,9.0,8.0,0,1,0,strict_14_with_grace_period,0,0
11734,32387679,https://www.airbnb.com/rooms/8569035,0.823108,3,32.70952,-117.15706,East Village,92101,Apartment,Entire home/apt,8,2.0,3.0,7,Real Bed,190.0,380.0,399.0,140.0,4,40.0,2,1125,within an hour,100.0,0,8.0,1,0,26,52,92.0,9.0,10.0,9.0,9.0,9.0,9.0,0,1,0,strict_14_with_grace_period,0,0
11820,36089401,https://www.airbnb.com/rooms/33068712,0.821092,1,32.71229,-117.16319,Marina,92101,Condominium,Entire home/apt,8,1.0,1.0,4,Real Bed,299.0,299.0,0.0,0.0,2,15.0,1,1125,within an hour,99.0,0,33.0,1,0,6,12,90.0,8.0,10.0,9.0,10.0,10.0,9.0,0,1,0,strict_14_with_grace_period,0,0
11815,31746305,https://www.airbnb.com/rooms/21385671,0.820378,3,32.71016,-117.15882,East Village,92101,Apartment,Entire home/apt,8,1.0,1.0,4,Real Bed,299.0,299.0,0.0,0.0,2,15.0,1,1125,within an hour,99.0,0,33.0,1,0,51,102,93.0,10.0,10.0,9.0,10.0,10.0,9.0,0,1,0,strict_14_with_grace_period,0,0


## Basitleştirilmiş Öneri Sistemi

###  `sd_trans` dataseti yüklüyoruz

In [78]:
# load in df with the cluster labels
sd_trans = pd.read_csv('data/sd_trans', index_col = 0)
sd_trans.head()

,neighbourhood_cleansed,zipcode,property_type,room_type,bed_type,host_response_time,cancellation_policy,accommodates,bathrooms,bedrooms,beds,nightly_price,price_per_stay,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,host_response_rate,host_is_superhost,host_total_listings_count,host_has_profile_pic,host_identity_verified,number_of_reviews,number_of_stays,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification
0,Scripps Ranch,92131,House,Private room,Real Bed,within a few hours,strict_14_with_grace_period,1,1.0,1.0,1,1100.0,1100.0,1000.0,0.0,1,0.0,60,90,100.0,0,1.0,1,0,0,0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0,1,0,0,0
1,La Jolla,92037,Apartment,Entire home/apt,Real Bed,no response,strict_14_with_grace_period,1,2.0,2.0,3,1400.0,1400.0,0.0,0.0,1,0.0,30,1125,0.0,0,1.0,1,0,2,4,100.0,10.0,10.0,10.0,10.0,10.0,10.0,0,1,0,0,0
2,Pacific Beach,92109,Townhouse,Private room,Real Bed,no response,flexible,1,1.0,1.0,1,1250.0,1250.0,0.0,0.0,1,0.0,30,30,0.0,0,1.0,1,0,0,0,97.0,10.0,10.0,10.0,10.0,10.0,10.0,0,1,0,0,0
3,College Area,92120,Apartment,Entire home/apt,Real Bed,no response,flexible,1,1.0,1.0,1,1150.0,1150.0,0.0,0.0,1,0.0,31,40,0.0,0,1.0,1,0,0,0,98.0,10.0,10.0,10.0,10.0,10.0,10.0,0,0,0,0,0
4,La Jolla,92037,Condominium,Private room,Real Bed,within a day,strict_14_with_grace_period,1,1.0,1.0,1,110.0,110.0,200.0,0.0,1,50.0,180,365,67.0,0,6.0,1,1,2,4,90.0,7.0,7.0,8.0,8.0,9.0,10.0,0,0,0,0,0


In [79]:
sd_trans.columns

Index(['neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type',
       'bed_type', 'host_response_time', 'cancellation_policy', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'nightly_price', 'price_per_stay',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'host_response_rate',
       'host_is_superhost', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified', 'number_of_reviews',
       'number_of_stays', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'requires_license', 'instant_bookable',
       'is_business_travel_ready', 'require_guest_profile_picture',
       'require_guest_phone_verification'],
      dtype='object')

In [80]:
features_to_keep = ['neighbourhood_cleansed', 'property_type', 'room_type',
                   'accommodates', 'bathrooms', 'beds', 'nightly_price', 'review_scores_rating']

sd_simplified = sd_trans[features_to_keep]
sd_simplified.head()

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
0,Scripps Ranch,House,Private room,1,1.0,1,1100.0,98.0
1,La Jolla,Apartment,Entire home/apt,1,2.0,3,1400.0,100.0
2,Pacific Beach,Townhouse,Private room,1,1.0,1,1250.0,97.0
3,College Area,Apartment,Entire home/apt,1,1.0,1,1150.0,98.0
4,La Jolla,Condominium,Private room,1,1.0,1,110.0,90.0


In [81]:
# save as csv
path = "deployment/"

sd_simplified.to_csv(path + 'sd_simplified')

### Yeni bir ön işleme pipeline oluştur

In [82]:
# define nominal and ordinal features in the categorical columns
nom_cols = sd_simplified.select_dtypes(['object']).columns
print(nom_cols)

ordinal_cols = sd_simplified.select_dtypes(['category']).columns
print(ordinal_cols)

Index(['neighbourhood_cleansed', 'property_type', 'room_type'], dtype='object')
Index([], dtype='object')


Bu basitleştirilmiş veri çerçevesinde yalnızca sayısal ve nominal özellikleri ön işleme yapmamız gerekiyor 

In [83]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Sayısal dönüşüm boru hattı
numeric_pipeline = Pipeline([
    ('numnorm', StandardScaler())
]) 

# Nominal dönüşüm boru hattı - Güncellenmiş OneHotEncoder
nominal_pipeline = Pipeline([
    ('onehotenc', OneHotEncoder(categories='auto', 
                                 sparse_output=False,  # 'sparse' yerine 'sparse_output' kullanıldı
                                 handle_unknown='ignore'))
])


In [84]:
#Seçilen sütunlar için pipelinelarla birlikte bir sütun dönüştürücü oluşturun
ct = ColumnTransformer(transformers = [("nominalpipe", nominal_pipeline, ['neighbourhood_cleansed', 
                                                                          'property_type', 'room_type']),
                                       ("numericpipe", numeric_pipeline, sd_simplified.select_dtypes(['int', 'float']).columns)])



In [85]:
# save the NEW column transformer
dump(ct, open('pkl/simple_column_transformer.pkl', 'wb'))

### Basitleştirilmiş veri setini dönüştürme

In [86]:
sd_simplified_pp = pd.DataFrame(ct.fit_transform(sd_simplified))

In [87]:
sd_simplified_pp.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.152178,-0.561234,-0.728205,2.546221,0.372801
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.152178,0.543694,0.324019,3.417967,0.683051
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.152178,-0.561234,-0.728205,2.982094,0.217675
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.152178,-0.561234,-0.728205,2.691512,0.372801
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.152178,-0.561234,-0.728205,-0.330541,-0.868203


### Custom bir listeleme veri çerçevesi oluştur.
Kullanıcı girdisini kullanarak benzer bir listeleme için önerilerde bulunun.

In [88]:
# Öneriler için değer girişi yapmak üzere yeni bir sütun içeren boş bir dataframe oluştur
input_df = pd.DataFrame(columns = sd_simplified.columns)

In [89]:
# Her bir prediktör için sayısal değer girin, değeri tam sayıya (int) dönüştürün
neighborhood_input = input("Enter Neighborhood: ")
property_input = input("Enter Property Type: ")
room_input = input("Enter Room Type: ")
accommodates = int(input("Enter Accommodation Number: "))
bathrooms = int(input("Number of Bathrooms: "))
beds = int(input("Number of Beds: "))
price = int(input("Nightly Price ($): "))
rating = int(input("Review Score Rating: "))

# Girdi değerlerini `input_df` sütunlarına atayın.
input_df.loc[0, 'neighbourhood_cleansed' ] = neighborhood_input
input_df.loc[0, 'property_type'] = property_input
input_df.loc[0, 'room_type' ] = room_input
input_df.loc[0, 'accommodates'] = accommodates
input_df.loc[0, 'bathrooms' ] = bathrooms
input_df.loc[0, 'beds' ] = beds
input_df.loc[0, 'nightly_price'] = price
input_df.loc[0, 'review_scores_rating' ] = rating

# convert to type float, int
input_df = input_df.astype({"accommodates":"int",
                            "bathrooms":"float",
                            "beds":"int",
                            "nightly_price":"float",
                            "review_scores_rating":"float"})

input_df

,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
0,La Jolla,House,Entire home/apt,3,2.0,2,250.0,90.0


### Basitleştirilmiş Veri Seti için Öneriler Alma

In [90]:
def get_simplified_recommendations(df, listing):
    """
   Ön işlenmiş veri çerçevesi ve ön işlenmiş özel listeyi giriş olarak alır 
   ve kosinüs benzerliğine dayalı olarak en iyi 5 öneriyi verir.
    """
    # reset the index
    df = df.reset_index(drop = 'index')
    
    # preprocess the listing input
    listing_pp = pd.DataFrame(ct.transform(listing))
    
    # convert single listing to an array
    listing_array = listing_pp.values
    # convert all listings to an array
    df_array = df.values
    
    # get arrays into a single dimension
    A = np.squeeze(np.asarray(df_array))
    B = np.squeeze(np.asarray(listing_array))
    
    # compute cosine similarity 
    cosine = np.dot(A,B)/(norm(A, axis = 1)*norm(B))
    
    # add similarity into recommendations df and reset the index
    rec = sd_simplified.copy().reset_index(drop = 'index')
    rec['similarity'] = pd.DataFrame(cosine).values
    
    # add in listings_urls
    # merge on index
    rec = rec.join(sd_listings_url)
    
    # reorder column names
    rec = rec[['listing_url', 'similarity', 'neighbourhood_cleansed', 'property_type', 
               'room_type', 'accommodates', 'bathrooms', 'beds', 'nightly_price', 'review_scores_rating']]
    
    # sort by top 5 descending
    return rec.sort_values(by = ['similarity'], ascending = False).head(5)


In [91]:
#Kullanıcı girişine en benzer 5 listeyi al.

get_simplified_recommendations(sd_simplified_pp, input_df)

,listing_url,similarity,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,beds,nightly_price,review_scores_rating
7542,https://www.airbnb.com/rooms/17975341,0.927941,La Jolla,House,Entire home/apt,4,2.0,2,187.0,84.0
6299,https://www.airbnb.com/rooms/25806580,0.908474,La Jolla,House,Entire home/apt,4,2.0,2,175.0,95.0
8666,https://www.airbnb.com/rooms/17975477,0.894235,La Jolla,House,Entire home/apt,5,2.0,2,411.0,93.0
9353,https://www.airbnb.com/rooms/22479052,0.871213,La Jolla,House,Entire home/apt,6,2.0,2,375.0,87.0
5244,https://www.airbnb.com/rooms/17975358,0.855470,La Jolla,House,Entire home/apt,3,1.0,2,185.0,90.0
